# Calculating accessibility for Competing Destination model

In [1]:
from spint.flow_accessibility import generate_dummy_flows
from spint.flow_accessibility import Accessibility

In [2]:
# Importing data
flow = generate_dummy_flows()
flow.head()

,origin_ID,destination_ID,volume_in_unipartite,volume_in_bipirtate,distances,nodes,dest_masses,results_all=False
0,A,A,10,0,0,A,60,500
1,A,B,10,0,8,B,10,510
2,A,C,10,10,2,C,10,730
3,A,D,10,10,5,D,30,230
4,A,E,10,10,5,E,50,190


## Data preparation

We imported data designed already the way it is most suitable for the function, however, this is not the same as the data we usually get from real sources. 

To demonstrate how should be data manipulated, let's create simple example.

Your best friend Henry decided to visit his grand parents over the easter. They live in 2 different cities. He needs to leave city A, travel to city B (20km away) to visit one of his grandparents, then travel to city C (30km from city A) to visit the other grandparents, and then travel back to city A (25km away from city B). This translate to 3 flows.

| Origin | Destination |
|----|---|
| A | B |
| B | C |
| C | A |

We know that he made one journwy between each of the cities.

| Origin | Destination | Journey/weight |
|----|---|---| 
| A | B | 1 |
| B | C | 1 |
| C | A | 1 |

Nevertheless, because accessibility looks at the distances between all the destinations, we need to construct all the possible flows in the network.
You can use [itertools](https://www.delftstack.com/howto/numpy/python-numpy-combinations/) to create all possible unique combinations of origins and destinations, and then just left-merge all the other data in.

| Origin | Destination | Journey/weight |
|----|---|---| 
| A | A | 0 |
| A | B | 1 |
| A | C | 0 |
| B | A | 0 |
| B | B | 0 |
| B | C | 1 |
| C | A | 1 |
| C | B | 0 |
| C | C | 0 |


For the accessibility calculation we also need to add the distance between the cities.


| Origin | Destination | Journey/weight | Distance |
|----|---|---|---|
| A | A | 0 | 0 |
| A | B | 1 | 20 |
| A | C | 0 | 25 |
| B | A | 0 | 20 |
| B | B | 0 | 0 |
| B | C | 1 | 30 |
| C | A | 1 | 25 |
| C | B | 0 | 30 |
| C | C | 0 | 0 |


The only thing missing now is masses.

| City | Mass | 
|----|---|
| A | 5000 |
| B | 1000 |
| C | 500 |

For the accessibility calculation we require the mass of the destination only, so let's add that in.

| Origin | Destination | Journey/weight | Distance | Destination Masses |
|----|---|---|---|---|
| A | A | 0 | 0 | 5000 |
| A | B | 1 | 20 | 1000 |
| A | C | 0 | 25 | 500 |
| B | A | 0 | 20 | 5000 |
| B | B | 0 | 0 | 1000 |
| B | C | 1 | 30 | 500 |
| C | A | 1 | 25 | 5000 |
| C | B | 0 | 30 | 1000 |
| C | C | 0 | 0 | 500 |

Thats it! You can calculate the accessibility by feeding this into the accessibility function.
Please use .loc to make sure that the columns are in the correct order as follows;
1. Origins
2. Destinations
3. Distances
4. Weight
5. Destination Masses

## Application

### Parameters

If *'all_destinations=False'* then only those destinations that recieve at least some flow from origin (weight >0) are considered.

If *'all_destinations=True'* then all destinations from origin are considered.

In [3]:
  
# apply accessibility to all data
flow['accessibility'] = Accessibility(flow_df = flow.loc[:,['origin_ID', 'destination_ID','distances', 'volume_in_unipartite','dest_masses']], all_destinations=False)

time elapsed: 0.08253149999999465


In [4]:
flow.iloc[:,[0,1,2,3,4,5,6,8]].head()

,origin_ID,destination_ID,volume_in_unipartite,volume_in_bipirtate,distances,nodes,dest_masses,accessibility
0,A,A,10,0,0,A,60,500
1,A,B,10,0,8,B,10,510
2,A,C,10,10,2,C,10,730
3,A,D,10,10,5,D,30,230
4,A,E,10,10,5,E,50,190
